In [0]:
#@title Copyright 2020 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_asset_from_cloud_geotiff.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_asset_from_cloud_geotiff.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Cloud GeoTiff Backed Earth Engine Assets

***Note:*** *The REST API contains new and advanced features that may not be suitable for all users.  If you are new to Earth Engine, please get started with the [JavaScript guide](https://developers.google.com/earth-engine/getstarted).*

Earth Engine can load images from Cloud Optimized GeoTiffs (COGs) in Google Cloud Storage ([learn more](https://developers.google.com/earth-engine/image_overview#images-from-cloud-geotiffs)).  This notebook demonstrates how to create Earth Engine assets backed by COGs.  An advantage of COG-backed assets is that the spatial and metadata fields of the image will be indexed at asset creation time, making the image more performant in collections.  (In contrast, an image created through `ee.Image.loadGeoTIFF` and put into a collection will require a read of the GeoTiff for filtering operations on the collection.)  A disadvantage of COG-backed assets is that they may be several times slower than standard assets when used in computations.

To create a COG-backed asset, make a `POST` request to the Earth Engine [`CreateAsset` endpoint](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.assets/create).  As shown in the following, this request must be authorized to create an asset in your user folder.

## Start an authorized session

To be able to make an Earth Engine asset in your user folder, you need to be able to authenticate as you when you make the request.  The Earth Engine Python authenticator can be leveraged as a client app that is able to pass your credentials along.  Follow the instructions in the cell output to authenticate.  (Note that this auth flow is not supported if this notebook is being run in playgroud mode; make a copy before proceeding).

For more details, see [this guide](https://google-auth.readthedocs.io/en/latest/user-guide.html#obtaining-credentials) on obtaining credentials in this manner, [this reference](https://google-auth-oauthlib.readthedocs.io/en/latest/reference/google_auth_oauthlib.flow.html) on the `Flow` module, [this reference](https://github.com/googleapis/google-api-python-client/blob/master/docs/client-secrets.md) for the client secrets format, and [`oauth.py`](https://github.com/google/earthengine-api/blob/master/python/ee/oauth.py) from the Earth Engine python library.

In [0]:
# This has details about the Earth Engine Python Authenticator client.
from ee import oauth
from google_auth_oauthlib.flow import Flow
import json

# Build the `client_secrets.json` file by borrowing the
# Earth Engine python authenticator.
client_secrets = {
    'web': {
        'client_id': oauth.CLIENT_ID,
        'client_secret': oauth.CLIENT_SECRET,
        'redirect_uris': [oauth.REDIRECT_URI],
        'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
        'token_uri': 'https://accounts.google.com/o/oauth2/token'
    }
}

# Write to a json file.
client_secrets_file = 'client_secrets.json'
with open(client_secrets_file, 'w') as f:
  json.dump(client_secrets, f, indent=2)

# Start the flow using the client_secrets.json file.
flow = Flow.from_client_secrets_file(client_secrets_file,
    scopes=oauth.SCOPES,
    redirect_uri=oauth.REDIRECT_URI)

# Get the authorization URL from the flow.
auth_url, _ = flow.authorization_url(prompt='consent')

# Print instructions to go to the authorization URL.
oauth._display_auth_instructions_with_print(auth_url)
print('\n')

# The user will get an authorization code.
# This code is used to get the access token.
code = input('Enter the authorization code: \n')
flow.fetch_token(code=code)

# Get an authorized session from the flow.
session = flow.authorized_session()

## Request body

The request body is an instance of an [EarthEngineAsset](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.assets#EarthEngineAsset).  This is where the path to the COG is specified, along with other useful properties.  Note that the image is a small area exported from the composite made in [this example script](https://code.earthengine.google.com/?scriptPath=Examples%3ACloud%20Masking%2FSentinel2).  See [this doc](https://developers.google.com/earth-engine/exporting#configuration-parameters) for details on exporting a COG.

Earth Engine will determine the bands, geometry, and other relevant information from the metadata of the TIFF. The only other fields that are accepted when creating a COG-backed asset are `properties`, `start_time`, and `end_time`.

In [0]:
# Request body as a dictionary.
request = {
  'type': 'IMAGE',
  'gcs_location': {
    'uris': ['gs://ee-docs-demos/COG_demo.tif']
  },
  'properties': {
    'source': 'https://code.earthengine.google.com/d541cf8b268b2f9d8f834c255698201d'
  },
  'startTime': '2016-01-01T00:00:00.000000000Z',
  'endTime': '2016-12-31T15:01:23.000000000Z',
}

from pprint import pprint
pprint(json.dumps(request))

## Send the request

Make the POST request to the Earth Engine [`CreateAsset` endpoint](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.assets/create).


In [0]:
# Where Earth Engine assets are kept.
project_folder = 'earthengine-legacy'
# Your user folder name and new asset name.
asset_id = 'users/user_folder_name/asset_name'

url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'

response = session.post(
  url = url.format(project_folder, asset_id),
  data = json.dumps(request)
)

pprint(json.loads(response.content))

## Details on COG-backed assets

### Permissions
The ACLs of COG-backed Earth Engine assets and the underlying data are managed separately. If a COG-backed asset is shared in Earth Engine, it is the owner's responsibility to ensure that the data in GCS is shared with the same parties. If the data is not visible, Earth Engine will return an error of the form "Failed to load the GeoTIFF at `gs://my-bucket/my-object#123456`" (123456 is the generation of the object).

### Generations
When a COG-backed asset is created, Earth Engine reads the metadata of the TIFF in Cloud Storage and creates asset store entry. The URI associated with that entry must have a generation.  See the [object versioning docs](https://cloud.google.com/storage/docs/object-versioning) for details on generations. If a generation is specified (e.g., `gs://foo/bar#123`), Earth Engine will use it. If a generation is not specified, Earth Engine will use the latest generation of the object. 

That means that if the object in GCS is updated, Earth Engine will return a "Failed to load the GeoTIFF at `gs://my-bucket/my-object#123456`" error because the expected object no longer exists (unless the bucket enables  multiple object versions).  This policy is designed to keep metadata of the asset in sync with the metadata of the object.  

### Configuration
In terms of how a COG should be configured, the TIFF MUST be:

- Tiled, where the tile dimensions are either:
  - 16x16
  - 32x32
  - 64x64
  - 128x128
  - 256x256
  - 512x512
  - 1024x1024

- Arranged so that all IFDs are at the beginning.

For best performance:

- Use tile dimensions of 128x128 or 256x256.
- Include power of 2 overviews.

See [this page](https://github.com/cogeotiff/rio-cogeo#web-optimized-cog) for more details on an optimized configuration.